In [1]:
import pandas as pd
import os
import json
import random
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, MistralForCausalLM, MistralForSequenceClassification
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, set_peft_model_state_dict, TaskType, PeftModel
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
import wandb
import transformers
from datetime import datetime
from datasets import load_dataset
import argparse
from tqdm import tqdm
import pickle

In [2]:
decomposition = """You are an expert in Chemical Engineering with extensive knowledge of ionic liquids. Given the following claim and list of statements related to ionic liquids, indicate if each statement entails the claim. Also, on a scale of 1 to 5 (low to high), indicate your confidence in your response.

Claim: {claim}

Statements: 
{statements}

Format your response as a Python dictionary in the following format: {{'<statement number>': {{'entails': <True/False>, 'confidence': <score between 1 to 5>}}}}.
Do not generate anything extra."""


def make_decomposition_prompt(example):
    statements, labels = [], []
    for ix, i in enumerate(example["options"]):
        statements.append(f"{ix+1}. {i[1]}")
        labels.append(1 if i[0] >= 0 else 0)
    statements.append(f"{ix+2}. None of the above")
    labels.append(0 if sum(labels) > 0 else 1)
    statements = "\n".join(statements)

    return decomposition.format(claim=example["claim"], statements=statements), labels

def generate_decomposition_response(tokenizer, model, content, do_sample=False):
    chat = [{ "role": "user", "content": content }]
    prompt_ = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs_ = tokenizer.encode(prompt_, add_special_tokens=False, return_tensors="pt")
    outputs_ = model.generate(input_ids=inputs_.to(model.device), max_new_tokens=256, do_sample=False)
    return tokenizer.decode(outputs_[0])


In [3]:
decomposition_experiment_dict = pickle.load(open("../data/decomposition_experiment_dict.pkl", "rb"))

In [7]:
decomposition_experiment_dict.keys(), decomposition_experiment_dict["7_options"].keys()

(dict_keys(['5_options', '7_options', '10_options', '15_options']),
 dict_keys(['Exp_1', 'Exp_2', 'Exp_3', 'Exp_4', 'Exp_5', 'Exp_6', 'Exp_7', 'Exp_8', 'Exp_9', 'Exp_10', 'Exp_11', 'Exp_12', 'Exp_13', 'Exp_14', 'Exp_15', 'Exp_16', 'Exp_17', 'Exp_18', 'Exp_19', 'Exp_20']))

In [4]:
prompt, label = make_decomposition_prompt(decomposition_experiment_dict[f"{7}_options"][f"Exp_{9}"][33])
print(prompt,"\n\n", label)

You are an expert in Chemical Engineering with extensive knowledge of ionic liquids. Given the following claim and list of statements related to ionic liquids, indicate if each statement entails the claim. Also, on a scale of 1 to 5 (low to high), indicate your confidence in your response.

Claim: Higher energy consumption for regeneration of ionic liquids is a cost-intensive process.

Statements: 
1. Adding water to conventional ionic liquids can reduce their viscosity.
2. Ionic liquid regeneration involves energy consumption only during rainy seasons.
3. Energy consumption for ionic liquids regeneration incurs cost only during rainy seasons.
4. Imidazolium-based ionic liquids can have alkyl groups attached to the imidazolium ring.
5. Energy consumption for ionic liquids regeneration incurs cost.
6. The viscosity of task-specific ionic liquids can vary.
7. Ionic liquid regeneration involves energy consumption.
8. None of the above

Format your response as a Python dictionary in the fo

In [5]:
model_ids = {"llama": "meta-llama/Meta-Llama-3.1-8B-Instruct"}

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

op_dict = {}
for k,v in model_ids.items():
    print(f"Loading {k}!")
    op_dict[k] = {}    
    tokenizer = AutoTokenizer.from_pretrained(v)
    model = AutoModelForCausalLM.from_pretrained(v, attn_implementation="flash_attention_2",
                                                 quantization_config=bnb_config,
                                                 device_map="auto")
    print(f"{k} loaded! Running prompts!")
    for n_opts, n_exp in decomposition_experiment_dict.items():
        op_dict[k][n_opts] = {}
        for exp_num, examples in n_exp.items():
            o_, e_ = int(n_opts.split("_")[0]), int(exp_num.split("_")[-1])
            if (o_ < 15) | ((o_ == 15) & (e_ <= 16)): # Resuming after ../data/decomp-llama-15_options-Exp_16-DedicatedOutput.pkl
                pass
            else:
                op_dict[k][n_opts][exp_num] = []
                print(f"Running [{k}] for [{n_opts}] [{exp_num}]!")
                for example in tqdm(examples):
                    prompt, lbl = make_decomposition_prompt(example)
                    res = generate_decomposition_response(tokenizer, model, prompt, do_sample=False)
                    op_dict[k][n_opts][exp_num].append({"response": res, "label": lbl})
                fname = f"../data/decomp-{k}-{n_opts}-{exp_num}-DedicatedOutput.pkl"
                print(f"Saving interim results to: {fname}")                
                pickle.dump(op_dict, open(fname, "wb"))

print("All Done!")                
# pickle.dump(op_dict, open("../data/decomposition_experiment_allOutputs.pkl", "wb"))

Loading llama!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

llama loaded! Running prompts!
Running [llama] for [15_options] [Exp_17]!


  0%|                                                                                          | 0/74 [00:00<?, ?it/s]/home/csgrad/sougatas/anaconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/csgrad/sougatas/anaconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 27%|█████████████████████▉                                                           | 20/74 [05:12<14:01, 15.58s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 28%|██████████████████████▉                                                          | 21/74 [05:28<13:45, 15.57s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 30%|████████████████████████                                                         | 22/74 [05:43<13:30, 15.58s/it]The attention mask and the pad token id were not set. As a consequenc

...

  0%|                                                            | 0/74 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|▋                                                   | 1/74 [00:15<18:53, 15.53s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  3%|█▍                                                  | 2/74 [00:31<18:44, 15.62s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for o

 32%|████████████████▌                                  | 24/74 [06:13<12:59, 15.59s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 34%|█████████████████▏                                 | 25/74 [06:28<12:42, 15.55s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 35%|█████████████████▉                                 | 26/74 [06:44<12:27, 15.57s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for o

 65%|█████████████████████████████████                  | 48/74 [12:26<06:45, 15.59s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 66%|█████████████████████████████████▊                 | 49/74 [12:42<06:29, 15.60s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 68%|██████████████████████████████████▍                | 50/74 [12:58<06:14, 15.60s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for o

 97%|█████████████████████████████████████████████████▌ | 72/74 [18:38<00:30, 15.36s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 99%|██████████████████████████████████████████████████▎| 73/74 [18:54<00:15, 15.44s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|███████████████████████████████████████████████████| 74/74 [19:10<00:00, 15.54s/it]

Saving interim results to: ../data/decomp-llama-15_options-Exp_20-DedicatedOutput.pkl
All Done!


In [2]:
lbl

NameError: name 'lbl' is not defined

In [15]:
{1: {'entails': True, 'confidence': 5}, 2: {'entails': False, 'confidence': 5}, 3: {'entails': True, 'confidence': 5}, 4: {'entails': True, 'confidence': 5}, 5: {'entails': True, 'confidence': 5}, 6: {'entails': False, 'confidence': 5}}

{1: {'entails': True, 'confidence': 5},
 2: {'entails': False, 'confidence': 5},
 3: {'entails': True, 'confidence': 5},
 4: {'entails': True, 'confidence': 5},
 5: {'entails': True, 'confidence': 5},
 6: {'entails': False, 'confidence': 5}}